In [1]:
import numpy as np
G = np.array([
    [1, 0, 0],
    [0.5, 0.5, 0.5],
    [0.5, -0.5, 0.5],
    [0, 0, 1]
])

B = np.array([
    [1, 0, -1, 0],
    [0, 1, 1, 0],
    [0, -1, 1, 0],
    [0, 1, 0, -1]
])

A = np.array([
    [1, 1, 1, 0],
    [0, 1, -1, -1]
])


def winograd_f2x2_kernel(kernel):
    return G @ kernel @ G.T


def winograd_f2x2_input(tile):
    return B.T @ tile @ B


def winograd_output(Y):
    return A @ Y @ A.T 


def winograd_conv2d(input_feature, kernel):
    U = winograd_f2x2_kernel(kernel)
    output_shape = (input_feature.shape[0] - 2, input_feature.shape[1] - 2)
    output = np.zeros(output_shape)
    for i in range(0, input_feature.shape[0] - 3, 2):
        for j in range(0, input_feature.shape[1] - 3, 2):
            tile = input_feature[i:i + 4, j:j + 4]
            V = winograd_f2x2_input(tile)
            Y = U * V
            Z = winograd_output(Y)
            output[i:i + 2, j:j + 2] = Z

    return output


In [2]:
if __name__ == "__main__":
    input_feature = np.array([
        [1, 2, 3, 4, 5, 6],
        [7, 8, 9, 10, 11, 12],
        [13, 14, 15, 16, 17, 18],
        [19, 20, 21, 22, 23, 24],
        [25, 26, 27, 28, 29, 30],
        [31, 32, 33, 34, 35, 36]
    ])

    kernel = np.array([
        [0, 0, 0],
        [0, 1, 0],
        [0, 0, 0]
    ])

    output_feature = winograd_conv2d(input_feature, kernel)

    print("Input Feature Map:")
    print(input_feature)
    print("\nKernel:")
    print(kernel)
    print("\nOutput Feature Map:")
    print(output_feature)


Input Feature Map:
[[ 1  2  3  4  5  6]
 [ 7  8  9 10 11 12]
 [13 14 15 16 17 18]
 [19 20 21 22 23 24]
 [25 26 27 28 29 30]
 [31 32 33 34 35 36]]

Kernel:
[[0 0 0]
 [0 1 0]
 [0 0 0]]

Output Feature Map:
[[ 0.  -3.   0.  -3. ]
 [-0.5 18.5 -0.5 20.5]
 [ 0.  -3.   0.  -3. ]
 [-0.5 30.5 -0.5 32.5]]


**2D Convolution**

In [2]:
import time
import numpy as np

def conv2d(input_matrix, kernel):
    H, W = input_matrix.shape
    K, _ = kernel.shape
    output_size = H - K + 1
    output_matrix = np.zeros((output_size, output_size))

    for i in range(output_size):
        for j in range(output_size):
            region = input_matrix[i:i+K, j:j+K]
            output_matrix[i, j] = np.sum(region * kernel)

    return output_matrix

input_matrix = np.random.randint(0, 10, (6, 6))
kernel = np.random.randint(0, 5, (3, 3)) 

start_time = time.time()
output_matrix = conv2d(input_matrix, kernel)
end_time = time.time()

print("Input Matrix:\n", input_matrix)
print("Kernel:\n", kernel)
print("Output Matrix:\n", output_matrix)
print(f"Execution Time: {end_time - start_time:.6f} seconds")

Input Matrix:
 [[1 0 8 4 1 9]
 [8 5 4 0 7 2]
 [2 1 0 8 4 9]
 [1 2 4 7 0 7]
 [2 5 5 1 9 1]
 [4 6 7 1 5 9]]
Kernel:
 [[1 3 2]
 [4 4 1]
 [4 2 0]]
Output Matrix:
 [[ 83.  72.  61.  95.]
 [ 51.  45.  84. 110.]
 [ 39.  78.  98.  95.]
 [ 76. 107.  88.  76.]]
Execution Time: 0.020085 seconds
